In [1]:
import pandas as pd
import numpy as np
import math as mth
import matplotlib.pyplot as plt
from scipy.integrate import quad

In [2]:
# Define the blocking probability 
def B(a, n):
    Blocking_probability=1
    for i in range(1,n+1):
        Blocking_probability=a * Blocking_probability / (i + a * Blocking_probability)
    return Blocking_probability

In [4]:
# Define the first order derivative of B over a 
def B_prime_a(a, n):
    return ((n / a) - 1 + B(a,n)) * B(a,n)

In [5]:
# Define the first order derivative of B over n 

def B_prime_n(a, n):
    integrand = lambda z: np.exp(-a * z) * (1 + z)**n * np.log(1 + z)
    integral_result, _ = quad(integrand, 0, np.inf)
    return -a * B(a,n)**2 * integral_result


In [6]:
# Define the second order derivative of B over a 

def B_double_prime_a(a, n):
    term1 = (B_prime_a(a,n) - (n / a**2)) * B(a,n)
    term2 = (B_prime_a(a,n)**2) / B(a,n)
    return term1 + term2


In [8]:
# Define the second order derivative of B over n 

def B_double_prime_n(a, n):
    integrand1 = lambda z: np.exp(-a * z) * (1 + z)**n * np.log(1 + z)
    integral1, _ = quad(integrand1, 0, np.inf)
    
    integrand2 = lambda z: np.exp(-a * z) * (1 + z)**n * (np.log(1 + z))**2
    integral2, _ = quad(integrand2, 0, np.inf)
    
    term1 = 2 * a**2 * B(a,n)**3 * (integral1**2)
    term2 = a * B(a,n)**2 * integral2
    return term1 - term2


In [9]:
# Define the second order derivative of B over a and n 

def B_double_prime_a_n(a, n):
    term1 = (n / a - 1 + 2 * B(a,n)) * B_prime_n(a, n)
    term2 = B(a,n) / a
    return term1 + term2

In [ ]:
# The first order derivative of a over n

def first_derivation(a,n):
    return -1*B_prime_n(a,n)/B_prime_a(a,n)

In [11]:
# The second order derivative of a over n

def second_derivation(a,n):
    first_term=(B_double_prime_n(a,n)+B_double_prime_a_n(a,n)*first_derivation(a,n))*B_prime_a(a,n)
    second_term=(B_double_prime_a_n(a,n)+B_double_prime_a(a,n)*first_derivation(a,n))*B_prime_n(a,n)
    third_term=first_term-second_term
    return -1*third_term/(B_prime_a(a,n))**2

In [14]:
a_range=np.linspace(1,200,200)
n_range=np.arange(1,200,1)
results=np.zeros((len(a_range),len(n_range)))

for i_a,a in enumerate(a_range):
    for i_n,n in enumerate(n_range):
        results[i_a,i_n]=second_derivation(a,n)


C:\Users\mahyar.taheribavi\AppData\Local\Temp\ipykernel_26124\146033194.py:4: RuntimeWarning: overflow encountered in power
  integrand1 = lambda z: np.exp(-a * z) * (1 + z)**n * np.log(1 + z)
C:\Users\mahyar.taheribavi\AppData\Local\Temp\ipykernel_26124\146033194.py:4: RuntimeWarning: invalid value encountered in scalar multiply
  integrand1 = lambda z: np.exp(-a * z) * (1 + z)**n * np.log(1 + z)
C:\Users\mahyar.taheribavi\AppData\Local\Temp\ipykernel_26124\146033194.py:5: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  integral1, _ = quad(integrand1, 0, np.inf)
C:\Users\mahyar.taheribavi\AppData\Local\Temp\ipykernel_26124\146033194.py:7: RuntimeWarning: overflow encountered in power
  integrand2 = lambda z: np.exp(-a * z) * (1 + z)**n * (np.log(1 + z))**2
C:\Users\mahyar.taheribavi\AppData\Local\Temp\ipykernel_26124\146033194.py:7: RuntimeWarning: invalid value enco

In [ ]:
# This shows that sign of the second-order derivate is always positive
(results < 0).any().any()

False

In [18]:
df= pd.DataFrame(results, index=a_range, columns=n_range)
df.to_excel('Test_convexity.xlsx', sheet_name='Results')


In [19]:
df

,1,2,3,4,5,6,7,8,9,10,...,190,191,192,193,194,195,196,197,198,199
1.0,0.435886,0.163911,0.101740,0.074849,0.059450,0.049235,0.041874,0.036291,0.031906,0.028374,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,0.356711,0.129236,0.080181,0.060368,0.049612,0.042616,0.037518,0.033537,0.030296,0.027589,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,0.300834,0.105037,0.063994,0.048127,0.040010,0.035027,0.031529,0.028824,0.026595,0.024686,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,0.260256,0.088027,0.052480,0.039048,0.032449,0.028630,0.026112,0.024256,0.022760,0.021476,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,0.229488,0.075566,0.044104,0.032349,0.026697,0.023557,0.021614,0.020282,0.019271,0.018432,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196.0,0.010001,0.002538,0.001145,0.000654,0.000425,0.000299,0.000223,0.000174,0.000139,0.000115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197.0,0.009951,0.002525,0.001139,0.000650,0.000422,0.000298,0.000222,0.000173,0.000139,0.000114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198.0,0.009902,0.002512,0.001133,0.000647,0.000420,0.000296,0.000221,0.000172,0.000138,0.000113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199.0,0.009853,0.002500,0.001127,0.000644,0.000418,0.000295,0.000220,0.000171,0.000137,0.000113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
